In [ ]:
# db.py

from dotenv import load_dotenv
import os
import logging
import threading
import time
from typing import Tuple, List

from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents import Document

load_dotenv()
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configuration ---
DOCS_DIRECTORY = "."
HR_DOCUMENT_FILES = [
    "04_복지정책_v1.0.md"
]
# HR_DOCUMENT_FILES = [                 # 문서 추가 후, 변경
#     "01_직원핸드북_v1.0_2025-01-10.md",
#     "02_근무정책_v1.0.md",
#     "03_휴가정책_v1.0.md",
#     "04_복지정책_v1.0.md",
#     "05_장비·보안정책_v1.0.md",
# ]

EXISTING_HR_DOCS = [
    os.path.join(DOCS_DIRECTORY, f) for f in HR_DOCUMENT_FILES 
    if os.path.exists(os.path.join(DOCS_DIRECTORY, f))
]

# --- Caching ---
_VSTORE_CACHE = {}
_VSTORE_LOCK = threading.Lock()


# --- Pinecone Initialization ---
def _pc() -> Pinecone:
    """Pinecone 클라이언트 초기화 및 반환"""
    api_key = os.getenv("PINECONE_API_KEY")
    if not api_key:
        raise RuntimeError("PINECONE_API_KEY가 환경 변수에 설정되지 않았습니다.")
    return Pinecone(api_key=api_key)

def _index_exists(pc: Pinecone, name: str) -> bool:
    """Pinecone 인덱스 존재 여부 확인"""
    try:
        return name in pc.list_indexes().names()
    except Exception as e:
        logging.warning(f"인덱스 조회 중 오류 발생: {e}. Fallback 로직을 시도합니다.")
        try:
            return name in [idx['name'] for idx in pc.list_indexes()]
        except Exception as e_fallback:
            logging.error(f"Fallback 인덱스 조회도 실패했습니다: {e_fallback}")
            return False

def _ensure_index(pc: Pinecone, name: str, dimension: int) -> None:
    """Pinecone 인덱스가 없으면 생성하고 준비될 때까지 대기"""
    if _index_exists(pc, name):
        logging.info(f"Pinecone 인덱스 '{name}'가 이미 존재합니다.")
        return

    logging.info(f"Pinecone 인덱스 '{name}'를 생성합니다.")
    cloud = os.getenv("PINECONE_CLOUD", "aws")
    region = os.getenv("PINECONE_REGION", "us-east-1")

    pc.create_index(
        name=name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud=cloud, region=region),
    )

    while not pc.describe_index(name).status['ready']:
        logging.info(f"인덱스 '{name}' 생성 대기 중...")
        time.sleep(2)
    logging.info(f"Pinecone 인덱스 '{name}' 준비 완료.")


# --- Document Loading and Splitting ---
def _load_and_split_docs(file_paths: List[str]) -> List[Document]:
    """여러 마크다운 파일을 로드하고 구조적으로 분할하여 문서 청크 리스트 반환"""
    all_split_docs = []
    # HR 문서 구조에 맞는 헤더 정의
    headers_to_split_on = [
        ("#", "doc_title"),
        ("##", "main_category"),
        ("###", "sub_category"),
    ]
    # MarkdownHeaderTextSplitter로 구조적 분할
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=False
    )
    # 긴 섹션을 위한 추가 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )

    for file_path in file_paths:
        try:
            loader = TextLoader(file_path, encoding="utf-8")
            doc_text = loader.load()[0].page_content
            
            md_header_splits = markdown_splitter.split_text(doc_text)
            
            for doc in md_header_splits:
                doc.metadata["source"] = os.path.basename(file_path)

            splits = text_splitter.split_documents(md_header_splits)
            all_split_docs.extend(splits)
            logging.info(f"'{file_path}' 로드 및 분할 완료: {len(splits)}개 청크 생성.")
        except Exception as e:
            logging.error(f"'{file_path}' 처리 중 오류 발생: {e}")

    return all_split_docs


# --- VectorStore ---
def get_vectorstore(
    index_name: str = "gaida-hr-rules",
    recreate: bool = False,
) -> PineconeVectorStore:
    """
    Pinecone 벡터 저장소를 가져오거나 생성합니다.
    - 캐시된 인스턴스가 있으면 반환합니다.
    - recreate=True이면, 인덱스 내 문서를 모두 삭제하고 새로 업로드합니다.
    - DB가 비어있으면 자동으로 문서를 업로드합니다.
    """
    cache_key = index_name
    with _VSTORE_LOCK:
        if not recreate and cache_key in _VSTORE_CACHE:
            logging.info(f"캐시된 VectorStore 인스턴스를 반환합니다: {cache_key}")
            return _VSTORE_CACHE[cache_key]
    """
    임베딩 모델별 dimension
    OpenAI text-embedding-ada-002: 1536
    OpenAI text-embedding-3-small: 1536
    OpenAI text-embedding-3-large: 3072
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    dimension = 1536

    pc = _pc()
    _ensure_index(pc, index_name, dimension)
    index = pc.Index(index_name)

    vectorstore = PineconeVectorStore(
        index=index, embedding=embeddings
    )

    stats = index.describe_index_stats()
    vector_count = stats.get("total_vector_count", 0)

    if recreate or vector_count == 0:
        if recreate and vector_count > 0:
            logging.info(f"인덱스 '{index_name}'의 모든 벡터({vector_count}개)를 삭제합니다.")
            index.delete(delete_all=True)
        
        if EXISTING_HR_DOCS:
            logging.info(f"존재하는 문서 파일을 DB에 업로드합니다: {EXISTING_HR_DOCS}")
            split_docs = _load_and_split_docs(EXISTING_HR_DOCS)
            if split_docs:
                logging.info(f"총 {len(split_docs)}개 청크를 인덱스 '{index_name}'에 업로드합니다.")
                vectorstore.add_documents(documents=split_docs, batch_size=100)
        else:
            logging.warning("존재하는 HR 문서 파일이 없어 업로드를 건너뜁니다.")
    else:
        logging.info(f"인덱스 '{index_name}'에 {vector_count}개의 벡터가 이미 존재합니다. (재생성 원할 시 recreate=True)")

    with _VSTORE_LOCK:
        _VSTORE_CACHE[cache_key] = vectorstore
    return vectorstore

c:\Users\AY\Desktop\PROJECTS\HR-chatbot-development-project\venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [ ]:
# nodes.py로 이동
def get_retriever(
    vectorstore: PineconeVectorStore,
    k: int = 5,
) -> VectorStoreRetriever:
    """주어진 벡터 저장소에서 Retriever를 생성합니다."""
    return vectorstore.as_retriever(search_kwargs={"k": k})

In [ ]:
print(EXISTING_HR_DOCS)

['.\\04_복지정책_v1.0.md']


In [3]:
# ========================================
# Pinecone 벡터스토어 생성 테스트
# 노트북에서 실행
# ========================================

import os
import logging
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def test_pinecone_vectorstore():
    """Pinecone 벡터스토어 생성 및 상태 테스트"""
    
    print("🔍 Pinecone 벡터스토어 테스트 시작")
    print("=" * 60)
    
    try:
        # 1. 환경변수 확인
        print("\n1️⃣ 환경변수 확인")
        api_key = os.getenv("PINECONE_API_KEY")
        openai_key = os.getenv("OPENAI_API_KEY")
        
        if not api_key:
            print("❌ PINECONE_API_KEY가 설정되지 않았습니다.")
            return False
        if not openai_key:
            print("❌ OPENAI_API_KEY가 설정되지 않았습니다.")
            return False
            
        print(f"✅ PINECONE_API_KEY: {api_key[:8]}...{api_key[-4:]}")
        print(f"✅ OPENAI_API_KEY: {openai_key[:8]}...{openai_key[-4:]}")
        
        # 2. 파일 존재 확인
        print("\n2️⃣ HR 문서 파일 확인")
        print(f"✅ EXISTING_HR_DOCS: {EXISTING_HR_DOCS}")
        
        if not EXISTING_HR_DOCS:
            print("⚠️  문서 파일이 없습니다. 빈 벡터스토어가 생성됩니다.")
        
        # 3. 벡터스토어 생성 테스트
        print("\n3️⃣ 벡터스토어 생성 테스트")
        vectorstore = get_vectorstore(
            index_name="gaida-hr-rules-test",  # 테스트용 별도 인덱스
            recreate=False
        )
        print("✅ 벡터스토어 생성 성공!")
        
        # 4. Pinecone 인덱스 상태 확인
        print("\n4️⃣ Pinecone 인덱스 상태 확인")
        pc = _pc()
        index = pc.Index("gaida-hr-rules-test")
        stats = index.describe_index_stats()
        
        print(f"📊 인덱스 통계:")
        print(f"   - 총 벡터 수: {stats.get('total_vector_count', 0)}")
        print(f"   - 인덱스 풀차지: {stats.get('index_fullness', 0)}")
        print(f"   - 차원 수: {stats.get('dimension', 'N/A')}")
        
        # 5. 검색 테스트 (벡터가 있는 경우만)
        vector_count = stats.get('total_vector_count', 0)
        if vector_count > 0:
            print("\n5️⃣ 벡터 검색 테스트")
            retriever = get_retriever(vectorstore, k=3)
            
            # 테스트 쿼리들
            test_queries = [
                "휴가는 어떻게 신청하나요?",
                "복지 혜택에는 무엇이 있나요?",
                "회사 정책에 대해 알려주세요"
            ]
            
            for query in test_queries:
                try:
                    docs = retriever.get_relevant_documents(query)
                    print(f"\n🔍 쿼리: '{query}'")
                    print(f"   검색된 문서 수: {len(docs)}")
                    
                    if docs:
                        print(f"   첫 번째 문서:")
                        print(f"     - 출처: {docs[0].metadata.get('source', 'N/A')}")
                        print(f"     - 내용 미리보기: {docs[0].page_content[:100]}...")
                    
                except Exception as e:
                    print(f"   ❌ 검색 오류: {e}")
        else:
            print("\n5️⃣ 벡터 검색 테스트 건너뜀 (벡터 없음)")
        
        # 6. 네임스페이스 확인 (있는 경우)
        print("\n6️⃣ 네임스페이스 확인")
        namespaces = stats.get('namespaces', {})
        if namespaces:
            for ns_name, ns_stats in namespaces.items():
                print(f"   📁 네임스페이스 '{ns_name}': {ns_stats.get('vector_count', 0)}개 벡터")
        else:
            print("   📁 기본 네임스페이스만 사용 중")
        
        print("\n🎉 모든 테스트 완료!")
        return True
        
    except Exception as e:
        print(f"\n❌ 테스트 실패: {e}")
        print(f"오류 타입: {type(e).__name__}")
        return False


def test_vectorstore_operations():
    """벡터스토어 기본 연산 테스트"""
    
    print("\n" + "=" * 60)
    print("🔧 벡터스토어 기본 연산 테스트")
    print("=" * 60)
    
    try:
        # 테스트용 인덱스 생성
        vectorstore = get_vectorstore(
            index_name="gaida-hr-rules-test",
            recreate=False
        )
        
        # 유사도 검색 테스트
        print("\n1️⃣ 유사도 검색 테스트")
        results = vectorstore.similarity_search("휴가 정책", k=3)
        print(f"   검색 결과 수: {len(results)}")
        
        for i, doc in enumerate(results, 1):
            print(f"   결과 {i}:")
            print(f"     출처: {doc.metadata.get('source', 'N/A')}")
            print(f"     내용: {doc.page_content[:150]}...")
            print()
        
        # 점수와 함께 검색 테스트
        print("2️⃣ 점수 포함 검색 테스트")
        results_with_scores = vectorstore.similarity_search_with_score("직원 혜택", k=2)
        
        for i, (doc, score) in enumerate(results_with_scores, 1):
            print(f"   결과 {i} (점수: {score:.4f}):")
            print(f"     출처: {doc.metadata.get('source', 'N/A')}")
            print(f"     내용: {doc.page_content[:100]}...")
            print()
            
        return True
        
    except Exception as e:
        print(f"❌ 연산 테스트 실패: {e}")
        return False


def cleanup_test_index():
    """테스트용 인덱스 삭제 (선택적)"""
    
    print("\n" + "=" * 60)
    print("🗑️  테스트 인덱스 정리")
    print("=" * 60)
    
    try:
        pc = _pc()
        test_index_name = "gaida-hr-rules-test"
        
        if _index_exists(pc, test_index_name):
            response = input(f"테스트 인덱스 '{test_index_name}'를 삭제하시겠습니까? (y/N): ")
            if response.lower() == 'y':
                pc.delete_index(test_index_name)
                print(f"✅ 테스트 인덱스 '{test_index_name}' 삭제 완료")
            else:
                print(f"⏭️  테스트 인덱스 '{test_index_name}' 유지")
        else:
            print(f"ℹ️  테스트 인덱스 '{test_index_name}'가 존재하지 않습니다.")
            
    except Exception as e:
        print(f"❌ 인덱스 삭제 실패: {e}")


# ========================================
# 실행 함수
# ========================================

def run_all_tests():
    """모든 테스트 실행"""
    
    print("🚀 Pinecone 벡터스토어 종합 테스트 시작")
    print("현재 시간:", logging.Formatter().formatTime(logging.LogRecord(
        name="test", level=logging.INFO, pathname="", lineno=0, 
        msg="", args=(), exc_info=None)))
    
    # 기본 테스트
    success1 = test_pinecone_vectorstore()
    
    # 연산 테스트 (기본 테스트 성공 시에만)
    success2 = False
    if success1:
        success2 = test_vectorstore_operations()
    
    # 결과 요약
    print("\n" + "=" * 60)
    print("📋 테스트 결과 요약")
    print("=" * 60)
    print(f"✅ 벡터스토어 생성 테스트: {'성공' if success1 else '실패'}")
    print(f"✅ 기본 연산 테스트: {'성공' if success2 else '실패' if success1 else '건너뜀'}")
    
    if success1 and success2:
        print("\n🎉 모든 테스트 성공! 벡터스토어가 정상적으로 작동합니다.")
    else:
        print("\n⚠️  일부 테스트 실패. 로그를 확인해주세요.")
    
    # 정리 옵션 제공
    cleanup_test_index()


# ========================================
# 노트북에서 실행
# ========================================

if __name__ == "__main__":
    # 전체 테스트 실행
    run_all_tests()
    
    # 또는 개별 테스트만 실행하고 싶다면:
    # test_pinecone_vectorstore()
    # test_vectorstore_operations()

🚀 Pinecone 벡터스토어 종합 테스트 시작
현재 시간: 2025-09-24 15:59:07,809
🔍 Pinecone 벡터스토어 테스트 시작

1️⃣ 환경변수 확인
✅ PINECONE_API_KEY: pcsk_FVz...gfz5
✅ OPENAI_API_KEY: sk-proj-...7XAA

2️⃣ HR 문서 파일 확인
✅ EXISTING_HR_DOCS: ['.\\04_복지정책_v1.0.md']

3️⃣ 벡터스토어 생성 테스트


2025-09-24 15:59:15,640 - INFO - Pinecone 인덱스 'gaida-hr-rules-test'를 생성합니다.
2025-09-24 15:59:26,886 - INFO - Pinecone 인덱스 'gaida-hr-rules-test' 준비 완료.
2025-09-24 15:59:28,892 - INFO - 존재하는 문서 파일을 DB에 업로드합니다: ['.\\04_복지정책_v1.0.md']
2025-09-24 15:59:28,917 - INFO - '.\04_복지정책_v1.0.md' 로드 및 분할 완료: 15개 청크 생성.
2025-09-24 15:59:28,924 - INFO - 총 15개 청크를 인덱스 'gaida-hr-rules-test'에 업로드합니다.
2025-09-24 15:59:30,339 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


✅ 벡터스토어 생성 성공!

4️⃣ Pinecone 인덱스 상태 확인


2025-09-24 15:59:34,615 - INFO - 캐시된 VectorStore 인스턴스를 반환합니다: gaida-hr-rules-test


📊 인덱스 통계:
   - 총 벡터 수: 0
   - 인덱스 풀차지: 0.0
   - 차원 수: 1536

5️⃣ 벡터 검색 테스트 건너뜀 (벡터 없음)

6️⃣ 네임스페이스 확인
   📁 기본 네임스페이스만 사용 중

🎉 모든 테스트 완료!

🔧 벡터스토어 기본 연산 테스트

1️⃣ 유사도 검색 테스트


2025-09-24 15:59:35,428 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


   검색 결과 수: 0
2️⃣ 점수 포함 검색 테스트


2025-09-24 15:59:36,283 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"



📋 테스트 결과 요약
✅ 벡터스토어 생성 테스트: 성공
✅ 기본 연산 테스트: 성공

🎉 모든 테스트 성공! 벡터스토어가 정상적으로 작동합니다.

🗑️  테스트 인덱스 정리
✅ 테스트 인덱스 'gaida-hr-rules-test' 삭제 완료
